<a href="https://colab.research.google.com/github/vijaygwu/classideas/blob/main/ContextualRecommender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

class ComplexContextualRecommender(nn.Module):
    def __init__(self, num_users, num_items, num_contexts, user_embed_dim, item_embed_dim, context_embed_dim):
        super(ComplexContextualRecommender, self).__init__()

        # Embedding layers with different dimensions
        self.user_embedding = nn.Embedding(num_users, user_embed_dim)
        self.item_embedding = nn.Embedding(num_items, item_embed_dim)
        self.context_embedding = nn.Embedding(num_contexts, context_embed_dim)

        # Calculate total embeddings dimension
        total_embed_dim = user_embed_dim + item_embed_dim + context_embed_dim

        # Dense layers with InstanceNorm and Dropout
        self.fc1 = nn.Linear(total_embed_dim, 256)
        self.in1 = nn.InstanceNorm1d(256)
        self.fc2 = nn.Linear(256, 128)
        self.in2 = nn.InstanceNorm1d(128)
        self.fc3 = nn.Linear(128, 64)
        self.in3 = nn.InstanceNorm1d(64)
        self.fc_out = nn.Linear(64, 1)  # Output a score
        self.dropout = nn.Dropout(0.5)

    def forward(self, user, item, context):
        user_embed = self.user_embedding(user)
        item_embed = self.item_embedding(item)
        context_embed = self.context_embedding(context)

        # Concatenate the embeddings
        x = torch.cat([user_embed, item_embed, context_embed], 1)

        # Pass through dense layers with activations, instance normalization, and dropout
        x = self.dropout(F.relu(self.in1(self.fc1(x).unsqueeze(1)).squeeze(1)))
        x = self.dropout(F.relu(self.in2(self.fc2(x).unsqueeze(1)).squeeze(1)))
        x = self.dropout(F.relu(self.in3(self.fc3(x).unsqueeze(1)).squeeze(1)))
        x = self.fc_out(x)
        return x


# Hyperparameters and Data
num_users = 1000
num_items = 5000
num_contexts = 3
user_embed_dim = 50
item_embed_dim = 100
context_embed_dim = 10
batch_size = 32

# Initialize the complex model
model = ComplexContextualRecommender(num_users, num_items, num_contexts, user_embed_dim, item_embed_dim, context_embed_dim)
criterion = nn.MSELoss() # Assuming a regression problem
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Generate random sample data
users = torch.randint(0, num_users, (1000,))
items = torch.randint(0, num_items, (1000,))
contexts = torch.randint(0, num_contexts, (1000,))
ratings = torch.rand((1000, 1)) # Random ratings


# Sample Training loop for 100 epochs
for epoch in range(100):
    model.train()
    total_loss = 0.0

    # Create mini-batches
    for i in range(0, len(users), batch_size):
        batch_users = users[i:i+batch_size]
        batch_items = items[i:i+batch_size]
        batch_contexts = contexts[i:i+batch_size]
        batch_ratings = ratings[i:i+batch_size]

        # Skip the batch if size is 1
        if len(batch_users) == 1:
            continue

        optimizer.zero_grad()
        outputs = model(batch_users, batch_items, batch_contexts)
        loss = criterion(outputs, batch_ratings)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    if (epoch+1) % 10 == 0:
        avg_loss = total_loss / (len(users) // batch_size)
        print(f'Epoch [{epoch+1}/100], Loss: {avg_loss:.4f}')


# Example prediction
user = torch.tensor([5])
item = torch.tensor([100])
context = torch.tensor([2])
score = model(user, item, context)
print(f"Predicted Score: {score.item():.4f}")


Epoch [10/100], Loss: 0.0953
Epoch [20/100], Loss: 0.0733
Epoch [30/100], Loss: 0.0398
Epoch [40/100], Loss: 0.0275
Epoch [50/100], Loss: 0.0195
Epoch [60/100], Loss: 0.0143
Epoch [70/100], Loss: 0.0132
Epoch [80/100], Loss: 0.0109
Epoch [90/100], Loss: 0.0104
Epoch [100/100], Loss: 0.0101
Predicted Score: 0.2313
